In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [18]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd

sns.set_theme(style="white")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

CAMBIOS_GLOSAS = {
    "Región Del Bíobío": "Región del Bío-Bío",
    "Región Metropolitana de Santiago": "Región Metropolitana de Santiago",
    "Región del Libertador General Bernardo O'Higgins": "Región del Libertador Bernardo O'Higgins",
    "Región De Antofagasta": "Región de Antofagasta",
    "Región De Los Lagos": "Región de Los Lagos",
    "Región Del Maule": "Región del Maule",
    "Región De Valparaíso": "Región de Valparaíso",
    "Región De Arica y Parinacota": "Región de Arica y Parinacota",
    "Región De Tarapacá": "Región de Tarapacá",
    "Región Aysén del General Carlos Ibáñez del Campo": "Región de Aysén del Gral.Ibañez del Campo",
    "Región De Coquimbo": "Región de Coquimbo",
    "Región De Magallanes y de La Antártica Chilena": "Región de Magallanes y Antártica Chilena",
    "Región De La Araucanía": "Región de La Araucanía",
    "Región De Atacama": "Región de Atacama",
    "Región De Ñuble": "Región de Ñuble",
    "Región De Los Ríos": "Región de Los Ríos",
}

In [19]:
df = pd.read_csv("../data/processed/df_procesada.csv")

In [23]:
# Eliminamos las filas que no tienen fecha de evaluación
df_limpia = df.dropna(subset="fecha_1_evaluacion").copy()
df_limpia = df_limpia.query("anio_1_evaluacion >= 2000")

df_limpia["region"] = df_limpia["region"].str.replace(", Chile", "")
df_limpia["region"] = df_limpia["region"].replace(CAMBIOS_GLOSAS)

df_limpia["fecha_1_evaluacion"] = pd.to_datetime(df_limpia["fecha_1_evaluacion"])
df_limpia["f_nac"] = pd.to_datetime(df_limpia["f_nac"])
df_limpia["edad_1_evaluacion_calc"] = (
    df_limpia["fecha_1_evaluacion"] - df_limpia["f_nac"]
).dt.days / 365

In [28]:
# Obtiene la cantidad de pacientes anuales
cantidad_de_pacientes = df_limpia.groupby("anio_1_evaluacion")["rut"].nunique()

# Obtiene la distribucion de la prevision a lo largo de los anios
prevision = df_limpia.groupby("anio_1_evaluacion")["prevision"].value_counts().unstack()
prevision.loc["Total", :] =  prevision.sum()

# Obtiene la distribucion de la 1era edad de consulta
edad_primera_consulta = df_limpia.groupby("anio_1_evaluacion")["edad_1_evaluacion_calc"].describe()

# Obtiene el desglose por region
desglose_por_region = df_limpia.groupby("region")["rut"].nunique()

In [29]:
desglose_por_region

region
Región Metropolitana de Santiago             1451
Región de Antofagasta                          61
Región de Arica y Parinacota                   58
Región de Atacama                              39
Región de Aysén del Gral.Ibañez del Campo      38
Región de Coquimbo                            219
Región de La Araucanía                         82
Región de Los Lagos                           151
Región de Los Ríos                             19
Región de Magallanes y Antártica Chilena       31
Región de Tarapacá                             46
Región de Valparaíso                          161
Región de Ñuble                                 7
Región del Bío-Bío                            229
Región del Libertador Bernardo O'Higgins      178
Región del Maule                              227
Name: rut, dtype: int64

In [7]:
map_df = gpd.read_file("../data/raw/mapas/Regional.shp")

In [30]:
desglose_por_region.to_frame().join(map_df.iloc[:, :7].set_index("Region"))

,rut,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_
region,,,,,,,
Región Metropolitana de Santiago,1451,1092,7,13,15392.030737,2.225204e+10,1.064253e+06
Región de Antofagasta,61,1086,3,2,126071.431981,1.508452e+11,2.516112e+06
Región de Arica y Parinacota,58,1084,1,15,16866.819844,1.886869e+10,7.505296e+05
Región de Atacama,39,1089,4,3,75661.248635,9.643906e+10,2.401741e+06
Región de Aysén del Gral.Ibañez del Campo,38,1088,14,11,106703.377369,2.242743e+11,4.144481e+07
Región de Coquimbo,219,1090,5,4,40575.901682,5.498082e+10,2.065933e+06
Región de La Araucanía,82,1095,11,9,31837.567634,5.221507e+10,1.501025e+06
Región de Los Lagos,151,1093,13,10,48408.371357,8.771834e+10,7.874158e+06
Región de Los Ríos,19,1094,12,14,18245.080563,3.108661e+10,1.844423e+06
